In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
np.set_printoptions(precision=3, suppress=True)
from tensorflow import keras
from tensorflow.keras import layers
import array as arr

In [ ]:
from ROOT import TCanvas, TGraph, TFile, TH1F, TH2D
import numpy as np
import re
import scipy as sc
import matplotlib.pyplot as plt

def collect_pulse_data(file):
    myFile = TFile.Open(file, "read")
    data = myFile.Get("PulseTransfer")
    detector = data.Get("detector1")
    pulsed_array = []
    for key in detector.GetListOfKeys():
        name = key.GetName()
        graph = detector.Get(name)
        if type(graph) != TGraph:
            continue
        else:
            searching = re.search("ke", graph.GetTitle())
            if searching != None and name[:10] == "current_ev":
                a = graph.GetTitle()[43:searching.start()]
                pulsed_array.append(a)
                event = detector.Get(name)
                x = event.GetX()
                y = event.GetY()
                x_title = event.GetXaxis()
                if (str(x_title))[22:-1] != 'ns':
                    print('Units have changed')
                    break
                
    return pulsed_array, x, y

def collect_efield(files_tot, position):
    each_field = arr.array('f', [])
    for file in files_tot:
        field_values = arr.array('f', [])
        for f, p in zip(file, position):
            myFile = TFile.Open(f, "read")
            data = myFile.Get("ElectricFieldReader")
            detector = data.Get("detector1")
            efield = detector.Get("field1d_z")
            no_bins = efield.GetNbinsX()
            bin_size = 0.1/no_bins
            
            pos = p*10**(-3)
                
            placement = -0.05
            for bin in range(no_bins):  
                placement += bin_size
                if placement >= pos:
                    field_strength = arr.array('f', [efield.GetBinContent(bin)])
                    field_values = np.concatenate((field_values, field_strength))
                    break
                    
        each_field = np.concatenate((each_field, field_values))
    each_field = each_field.reshape((len(files_tot), len(position), 1))
        
    return each_field

def array_data(field_x, field_y, time_vals):
    field_data = arr.array('f', [])
    for x_data, y_data in zip(field_x, field_y):
        data_tot = arr.array('f', [])
        for x, y in zip(x_data, y_data):
            y_array = arr.array('f', [])
            for t in time_vals:
                if x[-1] < t:
                    y_array.append(0)
                    if len(y_array) == len(time_vals):
                        data_tot = np.concatenate((data_tot, y_array))
                
                else:
                    for i in range(len(x)):
                        if x[i] >= t and x[i-1] < t:
                            y_array.append(y[i])
                            if len(y_array) == len(time_vals):
                                data_tot = np.concatenate((data_tot, y_array))
                            break
                        else:
                            continue
    
        field_data = np.concatenate((field_data, data_tot))
    field_data = field_data.reshape((len(field_x), len(x_data), len(time_vals)))

    return field_data

files1 = ["data_10ps_n40.root", "extra_data_n35.root", "data_10ps_n30.root", "extra_data_n25.root","data_10ps_n20.root", "extra_data_n15.root", "data_10ps_n10.root", "extra_data_n5.root", "data_10ps_0.root"] 
files2 = ["less_overbias_n40.root", "less_overbias_n35.root", "less_overbias_n30.root", "less_overbias_n25.root", "less_overbias_n20.root", "less_overbias_n15.root", "less_overboas_n10.root", "less_overbias_n5.root", "less_overbias_0.root"] 
positions = [40, 35, 30, 25, 20, 15, 10, 5, 0] 

x_values1, y_values1, pulse_names1 = [], [], []
for dat in files1:
    pulse, x_dat, y_dat = collect_pulse_data(dat)
    pulse_names1.append(pulse)
    x_values1.append(np.array(x_dat))
    y_values1.append(np.array(y_dat))

x_values2, y_values2, pulse_names2 = [], [], []
for dat in files2:
    pulse, x_dat, y_dat = collect_pulse_data(dat)
    pulse_names2.append(pulse)
    x_values2.append(np.array(x_dat))
    y_values2.append(np.array(y_dat))

x_val = [x_values1]+[x_values2]
y_val = [y_values1]+[y_values2]

data_tot = array_data(x_val, y_val, np.linspace(0.1,10,num=100))
print(data_tot)

files = [files1]+[files2]
e_field = collect_efield(files, positions)
print(e_field)

In [ ]:
column_names = ['Current_pulse', 'E_field']

x_train = data_tot.astype("float32")
y_train = e_field.astype("float32")

def build_and_compile_model():
  model = keras.Sequential([
    #layers.Embedding(input_dim=100, output_dim=100),
    layers.GRU(100, return_sequences=True),
    layers.SimpleRNN(100),
    layers.Dense(1000, activation='relu'),
    layers.Dense(1000, activation='relu'),
    layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

model = build_and_compile_model()

history = model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    verbose=0, epochs=500)


In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  #plt.ylim(0,1000)
  plt.xlabel('Epoch')
  plt.ylabel('Error')
  plt.legend()
  plt.grid(True)
  
plot_loss(history)
print(history.history['loss'])